# Evaluation of Time Parsings

In [1]:
from tf.app import use
import json

with open('../../../results/parsing/time_parsings.json', 'r') as infile:
    time_parsings = json.load(infile)

In [3]:
list(time_parsings.items())[:5]

[('427553',
  {'times': [2],
   'slots': [1, 2],
   'refs': [],
   'quals': ['point?'],
   'quants': [],
   'functions': ['simultaneous'],
   'preps': [['B', 1]]}),
 ('427680',
  {'times': [689],
   'slots': [687, 688, 689],
   'refs': [['THE', 688]],
   'quals': ['point?'],
   'quants': [],
   'functions': ['simultaneous'],
   'preps': [['B', 687]]}),
 ('427682',
  {'times': [699],
   'slots': [697, 698, 699],
   'refs': [['THE', 698]],
   'quals': ['point?'],
   'quants': [],
   'functions': ['simultaneous'],
   'preps': [['B', 697]]}),
 ('427693',
  {'times': [750],
   'slots': [750],
   'refs': [],
   'quals': ['point?'],
   'quants': [],
   'functions': ['anterior']}),
 ('427694',
  {'times': [760],
   'slots': [760],
   'refs': [],
   'quals': ['point?'],
   'quants': [],
   'functions': ['anterior']})]

# Build Experimental Notebook

I'd like to consider using notebooks as a way to display time taggings 
and edit their code manually. Notebooks provide the best way to blend the 
code of the time parsings with the HTML of the Hebrew TF objects I'd like to
visualize for the evaluation process.

In [4]:
import nbformat as nbf

<div class="section rtl"><a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Josua&amp;chapter=20&amp;verse=5&amp;version=c&amp;mr=m&amp;qw=q&amp;tp=txt_p&amp;tr=hb&amp;wget=v&amp;qget=v&amp;nget=vt" title="Show this on SHEBANQ" sec="Joshua 20:5">Joshua 20:5</a></div><div class="rtl children"><div class="contnr c2  rtl  " ><div class="lbl c2  " ><a href="#" class="nd">clause:450602</a></div><div class="children hor wrap rtl"><div class="contnr c1  rtl  " ><div class="lbl c1  " ><a href="#" class="nd">phrase:721414</a></div><div class="children hor wrap rtl"><div class="contnr c0 trm rtl  " ><div class="lbl c0 trm " ><a href="#" class="nd">124239</a> <a class=" txtu hbo" target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=c&amp;id=1W" title="Show this on SHEBANQ"><span class="txtu hbo">וְ</span></a></div><div class=\'features\'><span class="pdp xft" ><span class="f">pdp=</span>conj</span></div></div></div></div><div class="contnr c1  rtl  " ><div class="lbl c1  " ><a href="#" class="nd">phrase:721415</a></div><div class="children hor wrap rtl"><div class="contnr c0 trm rtl  " ><div class="lbl c0 trm " ><a href="#" class="nd">124240</a> <a class=" txtu hbo" target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=c&amp;id=1LA" title="Show this on SHEBANQ"><span class="txtu hbo">לֹֽא־</span></a></div><div class=\'features\'><span class="pdp xft" ><span class="f">pdp=</span>nega</span></div></div></div></div><div class="contnr c1  rtl  " ><div class="lbl c1  " ><a href="#" class="nd">phrase:721416</a></div><div class="children hor wrap rtl"><div class="contnr c0 trm rtl  " ><div class="lbl c0 trm " ><a href="#" class="nd">124241</a> <a class=" txtu hbo" target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=c&amp;id=1FNAv" title="Show this on SHEBANQ"><span class="txtu hbo">שֹׂנֵ֥א </span></a></div><div class=\'features\'><span class="pdp xft" ><span class="f">pdp=</span>verb</span><span class="st xft" ><span class="f">st=</span>a</span></div></div></div></div><div class="contnr c1  rtl  " ><div class="lbl c1  " ><a href="#" class="nd">phrase:721417</a></div><div class="children hor wrap rtl"><div class="contnr c0 trm rtl  " ><div class="lbl c0 trm " ><a href="#" class="nd">124242</a> <a class=" txtu hbo" target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=c&amp;id=1HWA" title="Show this on SHEBANQ"><span class="txtu hbo">ה֛וּא </span></a></div><div class=\'features\'><span class="pdp xft" ><span class="f">pdp=</span>prps</span></div></div></div></div><div class="contnr c1  rtl  " ><div class="lbl c1  " ><a href="#" class="nd">phrase:721418</a></div><div class="children hor wrap rtl"><div class="contnr c0 trm rtl  " ><div class="lbl c0 trm " ><a href="#" class="nd">124243</a> <a class=" txtu hbo" target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=c&amp;id=1L" title="Show this on SHEBANQ"><span class="txtu hbo">לֹ֖ו </span></a></div><div class=\'features\'><span class="pdp xft" ><span class="f">pdp=</span>prep</span></div></div></div></div><div class="contnr c1  rtl  " ><div class="lbl c1  " ><a href="#" class="nd">phrase:721419</a></div><div class="children hor wrap rtl"><div class="contnr c0 trm rtl  hl" ><div class="lbl c0 trm hl" ><a href="#" class="nd">124244</a> <a class=" txtu hbo" target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=c&amp;id=1MN" title="Show this on SHEBANQ"><span class="txtu hbo">מִ</span></a></div><div class=\'features\'><span class="pdp xft" ><span class="f">pdp=</span>prep</span></div></div><div class="contnr c0 trm rtl  hl" ><div class="lbl c0 trm hl" ><a href="#" class="nd">124245</a> <a class=" txtu hbo" target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=c&amp;id=1TMWLn" title="Show this on SHEBANQ"><span class="txtu hbo">תְּמֹ֥ול </span></a></div><div class=\'features\'><span class="pdp xft" ><span class="f">pdp=</span>subs</span><span class="st xft" ><span class="f">st=</span>a</span></div></div><div class="contnr c0 trm rtl  hl" ><div class="lbl c0 trm hl" ><a href="#" class="nd">124246</a> <a class=" txtu hbo" target="_blank" href="https://shebanq.ancient-data.org/hebrew/word?version=c&amp;id=1CLCWMn" title="Show this on SHEBANQ"><span class="txtu hbo">שִׁלְשֹֽׁום׃ </span></a></div><div class=\'features\'><span class="pdp xft" ><span class="f">pdp=</span>subs</span><span class="st xft" ><span class="f">st=</span>a</span></div></div></div></div></div></div></div>